In [48]:
from transformers import pipeline
import torch
import pandas as pd
from PIL import Image
import os
import evaluate
import math


torch.cuda.is_available()

True

In [75]:
# Prepare input samples

real_cat_dir = "archive/CAT_04"
tom_dir = "archive/tom_and_jerry/tom_and_jerry/tom"
real_dog_dir = "archive/n02110063-malamute"
cartoon_dog_dir = "archive/cartoon_dogs_val_images"

img_folder = real_dog_dir
images = []
files = []

for file in os.listdir(img_folder):
    try:
        img_path = os.path.join(img_folder, file)
        img = Image.open(img_path)
        images.append(img)
        files.append(file)
    except IOError:
        print(f"Could not read image: {file}")

In [76]:
# Prepare baseline models
classifier = pipeline(model="nateraw/vit-base-cats-vs-dogs",  device=0)

results = classifier(images)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [77]:
evaluation_results = []
sum_of_confidence = 0
for file, result in zip(files, results):
    print(f"Results for {file}: {result}")
    evaluation_results.append(int(result[0]["label"] == "dog"))
    sum_of_confidence += result[0]["score"]
print(f"Average confidence: {sum_of_confidence/len(evaluation_results)}")

Results for n02110063_18412.jpg: [{'label': 'dog', 'score': 0.9970868229866028}, {'label': 'cat', 'score': 0.0029131530318409204}]
Results for n02110063_5727.jpg: [{'label': 'dog', 'score': 0.9995123147964478}, {'label': 'cat', 'score': 0.00048766136751510203}]
Results for n02110063_16966.jpg: [{'label': 'dog', 'score': 0.9970268607139587}, {'label': 'cat', 'score': 0.0029731944669038057}]
Results for n02110063_938.jpg: [{'label': 'dog', 'score': 0.9994972944259644}, {'label': 'cat', 'score': 0.0005026537692174315}]
Results for n02110063_18663.jpg: [{'label': 'dog', 'score': 0.998339056968689}, {'label': 'cat', 'score': 0.0016609432641416788}]
Results for n02110063_16539.jpg: [{'label': 'dog', 'score': 0.99379563331604}, {'label': 'cat', 'score': 0.006204357370734215}]
Results for n02110063_18782.jpg: [{'label': 'dog', 'score': 0.9994533658027649}, {'label': 'cat', 'score': 0.0005466475849971175}]
Results for n02110063_11227.jpg: [{'label': 'dog', 'score': 0.9991238713264465}, {'label'

In [42]:
# Calculate the accuracy for cats and dogs
def calculate_accuracy(results):
    clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
    for ref, pred in zip(results, [1 for _ in range(len(results))]):
        clf_metrics.add(references=ref, predictions=pred)
    return clf_metrics.compute()   

In [78]:
calculate_accuracy(evaluation_results)

{'accuracy': 0.949438202247191,
 'f1': 0.9740634005763689,
 'precision': 0.949438202247191,
 'recall': 1.0}

In [27]:
# Prepare cluster models
# imagenet-1k
classifier = pipeline(model="JAW1453/swin-tiny-patch4-window7-224-DDD2", device=0)
classifier(test_img)

classifier = pipeline(model="nicolasdupuisroy/vit-gabor-detection-v2", device=0)
classifier(test_img)

[{'label': 'absent', 'score': 0.5122044682502747},
 {'label': 'present', 'score': 0.48779550194740295}]

In [28]:
# imagenet-21k
classifier = pipeline(model="alirzb/S1_M1_R1_beit_42507336", device=0)
classifier(test_img)

/opt/conda/lib/python3.11/site-packages/transformers/models/beit/feature_extraction_beit.py:28: FutureWarning: The class BeitFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use BeitImageProcessor instead.
  warnings.warn(


[{'label': 'None_seizures', 'score': 0.5324092507362366},
 {'label': 'Seizures', 'score': 0.46759071946144104}]